In [2]:
from time import time
start_time = time()

In [3]:

import pywren_ibm_cloud as pywren


In [4]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='11007eab-26fb-471c-a850-4f2bd4c721ca', project_access_token='p-f6bd793f68c5ad2805a05d46074d9077e76be212')
pc = project.project_context


## CB01.ENTRENAMIENTO.py

In [5]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_007c356678ae40188e2c58411b04f488 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='iHDdDIdO7M6NYJzQj4aDdD1emj2oKC1R-4SocY6sGCnZ',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_007c356678ae40188e2c58411b04f488.get_object(Bucket='montecarlo-donotdelete-pr-npk3qzjotb017k', Key='classify.gpkg')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 



In [6]:
from geopandas import read_file,GeoDataFrame
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier # este algoritmo es el que ofreci mejores resultados en las pruebas
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

sc = SparkContext.getOrCreate()
spark = SQLContext(sc)



In [7]:
datosEntrenamiento=read_file(streaming_body_1)

In [8]:
dfEntrenamiento=spark.createDataFrame(datosEntrenamiento)

In [9]:

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_2 = client_007c356678ae40188e2c58411b04f488.get_object(Bucket='montecarlo-donotdelete-pr-npk3qzjotb017k', Key='segmented_TopFixed.gpkg')['Body']
# add missing __iter__ method so pandas accepts body as file-like object
if not hasattr(streaming_body_2, "__iter__"): streaming_body_2.__iter__ = types.MethodType( __iter__, streaming_body_2 ) 


In [10]:
datosGlobales=read_file(streaming_body_2)

In [11]:
dfGlobal=spark.createDataFrame(datosGlobales)

In [12]:
dfE=dfEntrenamiento.select("cat","tipo").join(dfGlobal,"cat","inner")

In [13]:
columnas=['area', 'perimeter',  'fd', 'perimeter', 'compact_circle', 'fd', 'B1_sum', 'B1_mean', 'B1_median', 'B1_stdev', 'B1_min', 'B1_max', 'B1_variance', 'B2_sum', 'B2_mean', 'B2_median', 'B2_stdev', 'B2_min', 'B2_max', 'B2_variance', 'B3_sum', 'B3_mean', 'B3_median', 'B3_stdev', 'B3_min', 'B3_max', 'B3_variance']
constructor=VectorAssembler(inputCols=columnas,outputCol="features")


In [14]:
dfEF=constructor.transform(dfE).select("cat","features","tipo")

In [15]:
entrena,evalua=dfEF.randomSplit([0.8,0.2])

In [16]:
rf=RandomForestClassifier(labelCol="tipo")

In [17]:
modelo=rf.fit(entrena)

In [18]:
pred=modelo.transform(evalua)

In [19]:
evaluador=MulticlassClassificationEvaluator(labelCol="tipo",metricName="accuracy")

In [20]:
evaluador.evaluate(pred)

0.8157894736842105

In [21]:
modeloOk = rf.fit(dfEF)

In [22]:
modeloOk.write().overwrite().save('modeloRF')

In [23]:
#project.save_data(data = modeloOk, file_name = 'modeloRf', overwrite = True) 

In [24]:
import os

In [25]:
os.getcwd()

'/home/dsxuser/work'

In [26]:
os.listdir()

['modeloRF']

In [27]:
from pyspark.ml.classification import RandomForestClassificationModel
import pandas as pd

## CB02.PREDICCION.py

In [28]:
dfEF2=constructor.transform(dfGlobal).select("cat","features")

In [29]:
modelo=RandomForestClassificationModel().read().load("modeloRF")

In [30]:
pred=modelo.transform(dfEF2).select("cat","prediction")

In [31]:
predPandas=pd.merge(datosGlobales, pred.toPandas(), on='cat', how='inner')

In [32]:
geoPredPandas=GeoDataFrame(predPandas)

In [33]:
geoPredPandas.to_file("prediccion.gpkg",driver="GPKG")

## CB02.VALIDACION.py

In [34]:
from sklearn.model_selection import ShuffleSplit,cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np 

In [35]:
df=datosEntrenamiento[["cat","tipo"]].merge(datosGlobales,on="cat",how="left").dropna()

In [36]:
columnasVal=['area', 'perimeter',  'fd', 'compact_circle', 'B1_sum', 'B1_mean', 'B1_median', 'B1_stdev', 'B1_min', 'B1_max', 'B1_variance', 'B2_sum', 'B2_mean', 'B2_median', 'B2_stdev', 'B2_min', 'B2_max', 'B2_variance', 'B3_sum', 'B3_mean', 'B3_median', 'B3_stdev', 'B3_min', 'B3_max', 'B3_variance']



In [37]:
y=np.array(df['tipo'])
X=np.array(df[columnas])


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

In [39]:
clasificador=RandomForestClassifier(n_estimators=30)

In [40]:
cv = ShuffleSplit(n_splits=10,test_size=0.2,random_state=0)

In [41]:
evaluacion=cross_val_score(clasificador,X,y,cv=cv)

In [42]:
print("EN %d PREDICCIONES\nACIERTO media:  %0.2f, desviación típica: %0.2f" % (cv.n_splits,evaluacion.mean(),evaluacion.std()*2))

EN 10 PREDICCIONES
ACIERTO media:  0.79, desviación típica: 0.08


In [43]:
elapsed = time()
print("\nCompleted in: " + str(elapsed - start_time) + " seconds")


Completed in: 327.51116609573364 seconds


In [44]:
print(os.stat("prediccion.gpkg").st_size/1024/1024)

163.15234375


In [45]:
client_007c356678ae40188e2c58411b04f488.upload_file('prediccion.gpkg','montecarlo-donotdelete-pr-npk3qzjotb017k','prediccion.gpkg',ExtraArgs={'Metadata': {'mykey': 'iHDdDIdO7M6NYJzQj4aDdD1emj2oKC1R-4SocY6sGCnZ'}})
